In [1]:
import random
import tkinter as tk
from tkinter import messagebox, simpledialog  # Import simpledialog for category selection

# Function to load a word list from a file
def load_word_list(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    return words

# Function to choose a random word from a list
def choose_word(word_list):
    return random.choice(word_list)

# Function to display the current state of the word with guessed letters
def display_current_state(word, guessed_letters):
    return ' '.join([letter if letter in guessed_letters else '_' for letter in word])

# Function to load a word list by category
def load_word_list_by_category():
    return {
        'animals': ['elephant', 'tiger', 'giraffe'],
        'technology': ['computer', 'internet', 'python'],
        'food': ['pizza', 'burger', 'pasta']
    }

# Function to provide a hint to the player
def provide_hint(word, guessed_letters):
    unguessed_letters = [letter for letter in word if letter not in guessed_letters]
    if unguessed_letters:
        return random.choice(unguessed_letters)
    return ''

# Function to display the hangman stages based on incorrect guesses
def display_hangman(incorrect_guesses):
    hangman_stages = [
        """
         -----
         |   |
             |
             |
             |
             |
        ========
        """,
        """
         -----
         |   |
         O   |
             |
             |
             |
        ========
        """,
        """
         -----
         |   |
         O   |
         |   |
             |
             |
        ========
        """,
        """
         -----
         |   |
         O   |
        /|   |
             |
             |
        ========
        """,
        """
         -----
         |   |
         O   |
        /|\\  |
             |
             |
        ========
        """,
        """
         -----
         |   |
         O   |
        /|\\  |
        /    |
             |
        ========
        """,
        """
         -----
         |   |
         O   |
        /|\\  |
        / \\  |
             |
        ========
        """
    ]
    return hangman_stages[incorrect_guesses]

# Function to update the scoreboard
def update_scoreboard(scores, player, score):
    if player in scores:
        scores[player] += score
    else:
        scores[player] = score

# Function to display the scoreboard
def display_scoreboard(scores):
    print("\nScoreboard:")
    for player, score in scores.items():
        print(f"{player}: {score}")

# Hangman game class with tkinter interface
class HangmanGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Hangman Game")

        self.word_list = load_word_list_by_category()
        self.word = choose_word(self.word_list['animals'])  # Default to 'animals' category
        self.guessed_letters = set()
        self.incorrect_guesses = 0
        self.max_incorrect_guesses = 6
        self.hints_used = 0
        self.scores = {}
        self.player = ''

        self.create_widgets()
        self.update_display()

    def create_widgets(self):
        self.label_word = tk.Label(self.root, text='Current state: ' + display_current_state(self.word, self.guessed_letters))
        self.label_word.pack()

        self.entry_guess = tk.Entry(self.root)
        self.entry_guess.pack()

        self.button_guess = tk.Button(self.root, text="Guess", command=self.guess_letter)
        self.button_guess.pack()

        self.label_hangman = tk.Label(self.root, text=display_hangman(self.incorrect_guesses))
        self.label_hangman.pack()

        self.label_hint = tk.Label(self.root, text='')
        self.label_hint.pack()

        self.entry_player = tk.Entry(self.root)
        self.entry_player.pack()
        self.entry_player.insert(0, "Enter your name")  # Default text
        self.button_start = tk.Button(self.root, text="Start Game", command=self.start_game)
        self.button_start.pack()

    def start_game(self):
        self.player = self.entry_player.get().strip()
        if not self.player:
            messagebox.showerror("Error", "Please enter your name.")
            return

        category = simpledialog.askstring("Category", "Choose a category: animals, technology, food")
        if category not in self.word_list:
            messagebox.showerror("Error", "Invalid category. Defaulting to 'animals'.")
            category = 'animals'

        self.word = choose_word(self.word_list[category])
        self.guessed_letters = set()
        self.incorrect_guesses = 0
        self.hints_used = 0
        self.update_display()

    def guess_letter(self):
        guess = self.entry_guess.get().lower()
        if len(guess) != 1 or not guess.isalpha():
            messagebox.showerror("Error", "Invalid input. Please enter a single letter.")
            return

        if guess in self.guessed_letters:
            messagebox.showinfo("Info", "You've already guessed that letter.")
        elif guess in self.word:
            self.guessed_letters.add(guess)
            messagebox.showinfo("Info", "Good guess!")
        else:
            self.guessed_letters.add(guess)
            self.incorrect_guesses += 1
            messagebox.showinfo("Info", f"Incorrect guess! You have {self.max_incorrect_guesses - self.incorrect_guesses} guesses left.")

        if self.incorrect_guesses == 3 and self.hints_used == 0:
            hint = provide_hint(self.word, self.guessed_letters)
            if hint:
                self.label_hint.config(text=f"Hint: Try guessing the letter '{hint}'.")
                self.hints_used += 1

        if all(letter in self.guessed_letters for letter in self.word):
            messagebox.showinfo("Info", f"Congratulations! You've guessed the word: {self.word}")
            self.update_scoreboard(self.max_incorrect_guesses - self.incorrect_guesses)
            self.root.quit()
        elif self.incorrect_guesses >= self.max_incorrect_guesses:
            messagebox.showinfo("Info", f"Sorry, you've run out of guesses. The word was: {self.word}")
            self.update_scoreboard(0)
            self.root.quit()

        self.update_display()

    def update_display(self):
        self.label_word.config(text='Current state: ' + display_current_state(self.word, self.guessed_letters))
        self.label_hangman.config(text=display_hangman(self.incorrect_guesses))

    def update_scoreboard(self, score):
        if self.player in self.scores:
            self.scores[self.player] += score
        else:
            self.scores[self.player] = score

        display_scoreboard(self.scores)

def main():
    root = tk.Tk()
    game = HangmanGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()
